In [ ]:
!python -V

Python 3.8.10


In [ ]:
import os
import torch
import cv2
import numpy as np
import pandas as pd
from os import walk
from os.path import join
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
# connect to your google drive 
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/gdrive') 

In [ ]:
# clone git repositories in workspace
!git clone https://github.com/WongKinYiu/yolov7

# change directory to yolov7 file
%cd /content/yolov7

In [ ]:
'''
unzip the file HW1_material.zip which placed in your google drive ex: /content/gdrive/MyDrive/hw1/HW1_material.zip to /content/YoloV7
'''
!unzip {your_path}

## Task B
### Part1: Load a pretrained model and directly apply it to the `HW1_material/detect/parking_area.png` to detect the object.

In [ ]:
# download COCO starting checkpoint 
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
# run evaluation
!python detect.py --weights yolov7.pt --conf 0.1 --name parking_area --source HW1_material/detect/parking_area.png  --exist-ok

In [ ]:
# Show image in /content/yolov7/runs/detect/parking_area/parking_area.png
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('/content/yolov7/runs/detect/parking_area/parking_area.png')
imgplot = plt.imshow(img)
plt.show()

### Part2: Learn to fine tuning yolov7 model 

In [ ]:
# download COCO starting checkpoint 
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
# Finetune the Yolov7 model
'''
You can add other parameters by using --params_name params_values ex: --device 0 --epoch 1 --batch-size 1
just do not change parameters that we already give you.
hint: only one instruction 
'''
'''
Free to load another YoloV7 model that you can find here: https://github.com/WongKinYiu/yolov7
hint: only one instruction 

The best model will save in /content/yolov7/runs/train/yolov7-tiny/weights.
If you run this cell again then the model will save if /content/yolov7/runs/train/yolov7-tiny2/weight and so on.
'''
!python train.py --img-size 360 160 --data HW1_material/hw1.yaml --weights 'yolov7-tiny-custom' --exist-ok --cfg cfg/training/yolov7-tiny.yaml --name yolov7-tiny --hyp data/hyp.scratch.custom.yaml

In [ ]:
'''Save model in your google drive place'''
savePath = '{your google drive place/model.pt}'
!cp  '/content/yolov7/runs/train/yolov7-tiny/weights/best.pt' $savePath

In [ ]:
# If you don't have detect/train just ignore this remind (rm: cannot remove '/content/yolov7/runs/detect/train': No such file or directory)
!rm -r '/content/yolov7/runs/detect/train' 

# Apply on training data
''' 
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/train/car --name train --save-txt --exist-ok --weights $savePath 
hint: one instruction starting from !python 
'''
!python
''' 
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/train/non-car --name train --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python 
'''

!python

In [ ]:
def Calculate(AnswerFile,PredFile):

  def loadAns(dataPath):
      pos = os.path.join(dataPath, 'car')
      neg = os.path.join(dataPath, 'non-car')
      Answer = []
      for f in os.listdir(pos):
          if not os.path.isdir(f):
              Answer.append((os.path.join(pos, f), 1))
      for f in os.listdir(neg):
          if not os.path.isdir(f):
              Answer.append((os.path.join(neg, f), 0))
      return Answer

  Answer = loadAns(AnswerFile) 

  all_negatives, all_positives = 0, 0
  true_negatives, false_negatives = 0, 0
  true_positives, false_positives = 0, 0

  for x, y in Answer:

      exec_path = "'" + x + "'"
      img_filename = x.split('/')[-1]
      img_filename = img_filename.split('.')[0]
      txtpath = PredFile + img_filename + '.txt'

      if y == 1:
          all_positives += 1
      else:
          all_negatives += 1

      try:
          with open(txtpath, 'r') as f:
              n = len(f.readlines())
          if n > 0:
              prediction = 1
          else:
              prediction = 0
      except:
          prediction = 0

      if prediction == 1:
          if y == 1:
              true_positives += 1
          else:
              false_positives += 1
      else:
          if y == 1:
              false_negatives += 1
          else:
              true_negatives += 1

  correct = true_positives + true_negatives
  print("False Positive Rate: %d/%d (%f)" % (false_positives, all_negatives, false_positives/all_negatives))
  print("False Negative Rate: %d/%d (%f)" % (false_negatives, all_positives, false_negatives/all_positives))
  print("Training Accuracy: %d/%d (%f)" % (correct, len(Answer), correct/len(Answer)))

In [ ]:
# Calculate yolov7 performance 
# You have to adjust the parameters to get more than 90% accuracy
# Warning: make sure that txtpath is currect because you may get many train{n} file when training more than one time.

Calculate('/content/yolov7/HW1_material/train/','/content/yolov7/runs/detect/train/labels/')

In [ ]:
# If you don't have detect/test just ignore this remind (rm: cannot remove '/content/yolov7/runs/detect/test': No such file or directory)
!rm -r '/content/yolov7/runs/detect/test' 

# Apply on testing data
''' 
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/test/car --name test --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
hint: one instruction starting from !python 
'''
!python

''' 
Run evaluation like cell above
you must use parameter: --source /content/yolov7/HW1_material/test/non-car --name test --save-txt --exist-ok --weights $savePath
hint: one instruction starting from !python
hint: one instruction starting from !python 
'''
!python

In [ ]:
Calculate('/content/yolov7/HW1_material/test/','/content/yolov7/runs/detect/test/labels/')